In [36]:
import pandas as pd
import matplotlib.pyplot as plt
from process_score_calculator import  prepare_calculations as calculate_scores
from performance_analyzer import perform_analysis as analyze_performance

import warnings
warnings.filterwarnings("ignore")


In [37]:

scores_data = calculate_scores(MAC=True, DataName="BPIC_2019", layer="General_Process_Standards", index='case_id')

In [38]:
data = scores_data.data
data.head(2)

,eventID,cat_dim_5,cat_dim_4,cat_dim_3,cat_dim_2,case_Purchasing_Document,case_Purch._Doc._Category_name,cat_dim_8,cat_dim_6,cat_dim_7,...,activity,timestamp,case:concept:name,concept:name,time:timestamp,score_found_case_id,score_sequential_case_id,score_equilibrium_case_id,score_singular_case_id,score_exclusion_case_id
0,536870912000,Marketing,companyID_0000,EC Purchase order,Digital Marketing,2000000100,Purchase order,vendorID_0053,Service,"3-way match, invoice after GR",...,Vendor creates invoice,2017-01-31 23:59:00+00:00,2000000100_00001,Vendor creates invoice,2017-01-31 23:59:00+00:00,0.001112,0.0,0.000548,0.0,0.000894
1,536870912001,Marketing,companyID_0000,EC Purchase order,Digital Marketing,2000000100,Purchase order,vendorID_0053,Service,"3-way match, invoice after GR",...,Vendor creates invoice,2017-05-25 23:59:00+00:00,2000000100_00001,Vendor creates invoice,2017-05-25 23:59:00+00:00,0.001112,0.0,0.000548,0.0,0.000894


In [39]:
#score columns begin with score_
score_columns = [col for col in data.columns if 'score_' in col]
category_columns = [col for col in data.columns if 'category_' in col]
numeric_columns = [col for col in data.columns if 'num_' in col]

#create a Dataframe with only the score columns and case_id as index
scores = data[score_columns+['case_id']].set_index('case_id', drop=True)
scores.head(2)

,score_found_case_id,score_sequential_case_id,score_equilibrium_case_id,score_singular_case_id,score_exclusion_case_id
case_id,,,,,
2000000100_00001,0.001112,0.0,0.000548,0.0,0.000894
2000000100_00001,0.001112,0.0,0.000548,0.0,0.000894


In [40]:
%matplotlib inline

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def load_data():
    # Placeholder for your actual data loading function
    scores_data = calculate_scores(MAC=True, DataName="sample", layer="General_Process_Standards", index='case_id')
    return scores_data.data, scores_data.json

def get_score_and_category_columns(data):
    score_columns = [col for col in data.columns if col.startswith('score_')]
    category_columns = [col for col in data.columns if col.startswith('cat_')]
    return score_columns, category_columns

def extract_boxplot_stats(data, category, score):
    # Extract quartiles and outliers for a given score and category
    grouped = data.groupby(category)[score]
    q1 = grouped.quantile(0.25)
    q3 = grouped.quantile(0.75)
    iqr = q3 - q1
    median = grouped.median()
    lower_fence = q1 - 1.5 * iqr
    upper_fence = q3 + 1.5 * iqr
    
    # Identifying outliers
    outliers = data.apply(lambda x: x[score] if (x[score] < lower_fence[x[category]] or x[score] > upper_fence[x[category]]) else None, axis=1)
    
    # Building a DataFrame to hold the results
    stats_df = pd.DataFrame({
        'Lower Whisker': lower_fence,
        'Q1': q1,
        'Median': median,
        'Q3': q3,
        'Upper Whisker': upper_fence,
        'Outliers': outliers.groupby(data[category]).apply(list)
    })
    
    return stats_df

def save_stats_to_csv(data, category_columns, score_columns, file_prefix='boxplot_stats'):
    for category in category_columns:
        for score in score_columns:
            stats_df = extract_boxplot_stats(data, category, score)
            stats_df.to_csv(f'{file_prefix}_{category}_{score}.csv')

def create_seaborn_boxplots(data, category_columns, score_columns='all'):
    # Handling score columns selection
    if score_columns == 'all':
        score_columns = [col for col in data.columns if col.startswith('score_')]
    elif score_columns == 'mean':
        data['mean_score'] = data[[col for col in data.columns if col.startswith('score_')]].mean(axis=1)
        score_columns = ['mean_score']
    elif score_columns == 'sum':
        data['sum_score'] = data[[col for col in data.columns if col.startswith('score_')]].sum(axis=1)
        score_columns = ['sum_score']
    elif isinstance(score_columns, str):
        score_columns = [score_columns]

    sns.set(style="whitegrid")
    palette = sns.color_palette("muted")

    num_categories = len(category_columns)
    fig, axes = plt.subplots(nrows=num_categories, ncols=1, figsize=(15, 8 * num_categories), dpi=120)

    if num_categories == 1:
        axes = [axes]
    
    for ax, category in zip(axes, category_columns):
        for score in score_columns:
            sns.boxplot(x=category, y=score, data=data, ax=ax, palette=palette, hue=category)
            ax.set_title(f'Score Distribution for {score} by {category}', fontsize=12)
            ax.set_xlabel('Categories', fontsize=10)
            ax.set_ylabel('Scores', fontsize=10)
            ax.tick_params(axis='x', rotation=45)
            ax.legend(title=category, title_fontsize='13', fontsize='10', loc='upper right')
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

# Main function to orchestrate the steps
def main():
    data,json = load_data()
    score_columns, category_columns = get_score_and_category_columns(data)
    create_seaborn_boxplots(data, category_columns, score_columns='mean')
    save_stats_to_csv(data, category_columns, ['mean_score'])

# Run the main function
main()


AttributeError: 'ProcessScoreCalculator' object has no attribute 'json'

In [ ]:
#group by cat_dim_2 and sum all the columns beginning with "score_"
data_grouped = data.groupby("cat_dim_5").agg({col: 'sum' for col in data.columns if col.startswith("score_")})
data_grouped.head(20)

,score_found_case_id,score_sequential_case_id,score_equilibrium_case_id,score_singular_case_id,score_exclusion_case_id
cat_dim_5,,,,,
Additives,0.388383,0.447255,0.400310,0.385714,0.374364
CAPEX & SOCS,0.149517,0.157384,0.147398,0.142132,0.138646
Commodity Resins,0.016883,0.020445,0.018392,0.018674,0.017929
Energy,0.002779,0.018472,0.005482,0.010375,0.010151
Enterprise Services,0.011394,0.011692,0.010350,0.019712,0.018829
Latex & Monomers,0.161467,0.179820,0.174150,0.128423,0.147539
Logistics,0.845967,0.550459,0.417152,0.461591,0.772488
Marketing,0.551518,0.031478,0.286126,0.035274,0.449103
Others,0.031543,0.037184,0.032342,0.038386,0.037405


In [ ]:
#read the data from /Users/urszulajessen/code/gitHub/WISE/data/data_BPIC_2019/BPIC_2019.csv and filter it to only case_Item_Type == Service   
df = pd.read_csv("/Users/urszulajessen/code/gitHub/WISE/data/data_BPIC_2019/BPIC_2019.csv")
df = df[df["case_Item_Type"] == "Service"]
#save it to /Users/urszulajessen/code/gitHub/WISE/data/data_BPIC_2019_Service/BPIC_2019_Service.csv
df.to_csv("/Users/urszulajessen/code/gitHub/WISE/data/data_BPIC_2019_Service/BPIC_2019_Service.csv", index=False)


In [ ]:
#read the data from /Users/urszulajessen/code/gitHub/WISE/data/data_BPIC_2019/BPIC_2019.csv and get 50 random case_id
import pandas as pd
df_sample = pd.read_csv("/Users/urszulajessen/code/gitHub/WISE/data/data_BPIC_2019/BPIC_2019.csv")
random_case_id = df_sample["case_concept_name"].sample(n=50)
df_sample = df_sample[df_sample["case_concept_name"].isin(random_case_id)]
#save it to /Users/urszulajessen/code/gitHub/WISE/data/data_BPIC_2019_Service/BPIC_2019_Service.csv
df_sample.to_csv("/Users/urszulajessen/code/gitHub/WISE/data/data_sample/sample.csv", index=False)
#example to describe the data and Domain Knowledge -' Create Purchase Order Item only 1x but Record Goods Receipt 117 x and Vendor creates invoice 21x and clear invoice 29x
df_sample[df_sample["case_concept_name"] == "2000000100_00001"]

,eventID,case_Spend_area_text,case_Company,case_Document_Type,case_Sub_spend_area_text,case_Purchasing_Document,case_Purch._Doc._Category_name,case_Vendor,case_Item_Type,case_Item_Category,...,event_User,event_org_resource,event_concept_name,event_Cumulative_net_worth_(EUR),event_time_timestamp,@@index,@@case_index,case:concept:name,concept:name,time:timestamp
0,536870912000,Marketing,companyID_0000,EC Purchase order,Digital Marketing,2000000100,Purchase order,vendorID_0053,Service,"3-way match, invoice after GR",...,NONE,NONE,Vendor creates invoice,7815013.0,2017-01-31 23:59:00+00:00,0,0,2000000100_00001,Vendor creates invoice,2017-01-31 23:59:00+00:00
1,536870912001,Marketing,companyID_0000,EC Purchase order,Digital Marketing,2000000100,Purchase order,vendorID_0053,Service,"3-way match, invoice after GR",...,NONE,NONE,Vendor creates invoice,7815013.0,2017-05-25 23:59:00+00:00,1,0,2000000100_00001,Vendor creates invoice,2017-05-25 23:59:00+00:00
2,536870912209,Marketing,companyID_0000,EC Purchase order,Digital Marketing,2000000100,Purchase order,vendorID_0053,Service,"3-way match, invoice after GR",...,user_012,user_012,Record Invoice Receipt,7815013.0,2018-01-08 15:29:00+00:00,2,0,2000000100_00001,Record Invoice Receipt,2018-01-08 15:29:00+00:00
3,536870912296,Marketing,companyID_0000,EC Purchase order,Digital Marketing,2000000100,Purchase order,vendorID_0053,Service,"3-way match, invoice after GR",...,user_013,user_013,Record Invoice Receipt,7815013.0,2018-01-10 14:28:00+00:00,3,0,2000000100_00001,Record Invoice Receipt,2018-01-10 14:28:00+00:00
4,536870912297,Marketing,companyID_0000,EC Purchase order,Digital Marketing,2000000100,Purchase order,vendorID_0053,Service,"3-way match, invoice after GR",...,user_015,user_015,Cancel Invoice Receipt,7815013.0,2018-01-10 16:16:00+00:00,4,0,2000000100_00001,Cancel Invoice Receipt,2018-01-10 16:16:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,536870912457,Marketing,companyID_0000,EC Purchase order,Digital Marketing,2000000100,Purchase order,vendorID_0053,Service,"3-way match, invoice after GR",...,user_020,user_020,Record Invoice Receipt,7815013.0,2019-04-01 08:02:00+00:00,459,0,2000000100_00001,Record Invoice Receipt,2019-04-01 08:02:00+00:00
460,536870912458,Marketing,companyID_0000,EC Purchase order,Digital Marketing,2000000100,Purchase order,vendorID_0053,Service,"3-way match, invoice after GR",...,user_013,user_013,Record Invoice Receipt,7815013.0,2019-04-01 10:52:00+00:00,460,0,2000000100_00001,Record Invoice Receipt,2019-04-01 10:52:00+00:00
461,536870912459,Marketing,companyID_0000,EC Purchase order,Digital Marketing,2000000100,Purchase order,vendorID_0053,Service,"3-way match, invoice after GR",...,user_013,user_013,Record Invoice Receipt,7815013.0,2019-04-01 13:29:00+00:00,461,0,2000000100_00001,Record Invoice Receipt,2019-04-01 13:29:00+00:00
462,536870912463,Marketing,companyID_0000,EC Purchase order,Digital Marketing,2000000100,Purchase order,vendorID_0053,Service,"3-way match, invoice after GR",...,NONE,NONE,Vendor creates debit memo,7815013.0,2019-04-30 23:59:00+00:00,462,0,2000000100_00001,Vendor creates debit memo,2019-04-30 23:59:00+00:00


In [ ]:
df['concept:name'].value_counts()

concept:name
Record Service Entry Sheet             529
Record Goods Receipt                   392
Record Invoice Receipt                 150
Clear Invoice                           49
Vendor creates invoice                  43
Create Purchase Order Item              24
Cancel Invoice Receipt                  12
Vendor creates debit memo                8
Cancel Goods Receipt                     7
SRM: In Transfer to Execution Syst.      6
SRM: Ordered                             5
SRM: Document Completed                  5
SRM: Awaiting Approval                   5
SRM: Complete                            5
SRM: Created                             5
Remove Payment Block                     4
SRM: Deleted                             3
Change Price                             3
SRM: Change was Transmitted              2
SRM: Transfer Failed (E.Sys.)            1
Name: count, dtype: int64

In [ ]:
scores_data_Service = calculate_scores(MAC=True, DataName="BPIC_2019_Service", layer="General_Process_Standards").data
#group by cat_dim_2 and sum all the columns beginning with "score_" and add number of cases
data_grouped_service = scores_data_Service.groupby("cat_dim_5").agg({col: 'sum' for col in scores_data_Service.columns if col.startswith("score_")})
data_grouped_service["Number_of_cases"] = scores_data_Service.groupby("cat_dim_5").size()
data_grouped_service.head(20)

/Users/urszulajessen/code/gitHub/WISE/src/data_handler.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_occurrences['activity_order'] = first_occurrences.groupby('case_id').cumcount() + 1


,score_found,score_sequential,score_equilibrium,score_singular,score_exclusion,Number_of_cases
cat_dim_5,,,,,,
Logistics,752.0,-156.0,300.8,435.923077,752.000000,752
Marketing,475.0,-475.0,190.0,11.000000,419.761905,475
Sales,31.0,19.0,12.4,23.615385,31.000000,31


In [ ]:
#group df by case_Item_Type and pivot on concept:name with number of occurences for each
columns = ["Create Purchase Order Item", "Record Goods Receipt", "Record Service Entry Sheet"]
df_Service = df[df["concept:name"].isin(columns)]
df_grouped = df_Service.groupby("case_Spend_area_text")["concept:name"].value_counts().unstack().fillna(0)
df_grouped["Number_of_cases"] = df_Service.groupby("case_Spend_area_text").size()
df_grouped.head(20)


concept:name,Create Purchase Order Item,Record Goods Receipt,Record Service Entry Sheet,Number_of_cases
case_Spend_area_text,,,,
Logistics,18,270,407,695
Marketing,2,118,118,238
Sales,4,4,4,12


In [ ]:
df_Service.groupby("case_Spend_area_text").size()

case_Spend_area_text
Logistics    695
Marketing    238
Sales         12
dtype: int64

In [ ]:
df_Service

,eventID,case_Spend_area_text,case_Company,case_Document_Type,case_Sub_spend_area_text,case_Purchasing_Document,case_Purch._Doc._Category_name,case_Vendor,case_Item_Type,case_Item_Category,...,event_User,event_org_resource,event_concept_name,event_Cumulative_net_worth_(EUR),event_time_timestamp,@@index,@@case_index,case:concept:name,concept:name,time:timestamp
14,536870912009,Marketing,companyID_0000,EC Purchase order,Digital Marketing,2000000100,Purchase order,vendorID_0053,Service,"3-way match, invoice after GR",...,user_000,user_000,Create Purchase Order Item,7815013.0,2018-01-29 11:34:00+00:00,14,0,2000000100_00001,Create Purchase Order Item,2018-01-29 11:34:00+00:00
24,536870912013,Marketing,companyID_0000,EC Purchase order,Digital Marketing,2000000100,Purchase order,vendorID_0053,Service,"3-way match, invoice after GR",...,user_000,user_000,Record Goods Receipt,7815013.0,2018-02-16 16:20:00+00:00,24,0,2000000100_00001,Record Goods Receipt,2018-02-16 16:20:00+00:00
25,536870912014,Marketing,companyID_0000,EC Purchase order,Digital Marketing,2000000100,Purchase order,vendorID_0053,Service,"3-way match, invoice after GR",...,NONE,NONE,Record Service Entry Sheet,7815013.0,2018-02-16 16:20:00+00:00,25,0,2000000100_00001,Record Service Entry Sheet,2018-02-16 16:20:00+00:00
26,536870912015,Marketing,companyID_0000,EC Purchase order,Digital Marketing,2000000100,Purchase order,vendorID_0053,Service,"3-way match, invoice after GR",...,user_000,user_000,Record Goods Receipt,7815013.0,2018-02-16 16:23:00+00:00,26,0,2000000100_00001,Record Goods Receipt,2018-02-16 16:23:00+00:00
27,536870912016,Marketing,companyID_0000,EC Purchase order,Digital Marketing,2000000100,Purchase order,vendorID_0053,Service,"3-way match, invoice after GR",...,NONE,NONE,Record Service Entry Sheet,7815013.0,2018-02-16 16:23:00+00:00,27,0,2000000100_00001,Record Service Entry Sheet,2018-02-16 16:23:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6464,1049745841717328,Logistics,companyID_0000,Standard PO,Road Packed,4508073957,Purchase order,vendorID_0977,Service,"3-way match, invoice after GR",...,batch_02,batch_02,Record Goods Receipt,579.0,2018-12-12 23:56:00+00:00,6464,972,4508073957_00001,Record Goods Receipt,2018-12-12 23:56:00+00:00
6465,1049745841717334,Logistics,companyID_0000,Standard PO,Road Packed,4508073957,Purchase order,vendorID_0977,Service,"3-way match, invoice after GR",...,NONE,NONE,Record Service Entry Sheet,41.0,2018-12-12 23:56:00+00:00,6465,972,4508073957_00001,Record Service Entry Sheet,2018-12-12 23:56:00+00:00
6466,1049745841717336,Logistics,companyID_0000,Standard PO,Road Packed,4508073957,Purchase order,vendorID_0977,Service,"3-way match, invoice after GR",...,NONE,NONE,Record Service Entry Sheet,41.0,2018-12-12 23:56:00+00:00,6466,972,4508073957_00001,Record Service Entry Sheet,2018-12-12 23:56:00+00:00
6467,1049745841717337,Logistics,companyID_0000,Standard PO,Road Packed,4508073957,Purchase order,vendorID_0977,Service,"3-way match, invoice after GR",...,NONE,NONE,Record Service Entry Sheet,41.0,2018-12-12 23:56:00+00:00,6467,972,4508073957_00001,Record Service Entry Sheet,2018-12-12 23:56:00+00:00
